# 미션 1

In [27]:
import requests
import pandas as pd

# KEY 값
API_KEY = '5345624a6b617764353756767a474c'

# 서울시 부동산 실거래가 API는 경로기반(URL)으로만 작동함
TYPE = 'json'        # 요청 파일타입
START_INDEX = '1'    # 시작 페이지
END_INDEX = ' '   # 끝 페이지
RCPT_YR = ' '         # 접수연도
CGG_CD = ' '          # 자치구 코드(5자리 정수)
CGG_NM = ' '          # 자치구명(문자열)
STDG_CD = ' '         # 법정동코드(1:대지, 2:산, 3:블럭)
LOTNO_SE = ' '        # 지번구분(문자열)
LOTNO_SE_NM = ' '     # 지번구분명()
MNO = ' '             # 본번(4자리 정수)
SNO = ' '             # 부번(4자리 정수)
BLDG_NM = ' '         # 건물명(문자열)
CTRT_DAY = ' '        # 계약일(YYYYMMDD)
BLDG_USG = ' '        #건물용도(아파트/단독가구/연립다세대/오피스텔 택1)
# url = f'http://openapi.seoul.go.kr:8088/{API_KEY}/json/tbLnOpendataRtmsV/{start_index}/{end_index}/'
url = f'http://openapi.seoul.go.kr:8088/{API_KEY}/{TYPE}/tbLnOpendataRtmsV/{START_INDEX}/{END_INDEX}/{RCPT_YR}/{CGG_CD}/{CGG_NM}/{STDG_CD}/{LOTNO_SE}/{LOTNO_SE_NM}/{MNO}/{SNO}/{BLDG_NM}/{CTRT_DAY}/{BLDG_USG}'
res = requests.get(url)
data = res.json()

rows = data['tbLnOpendataRtmsV']['row']
df = pd.json_normalize(rows)

print(df.head(1))
# 강남구 필터
#df_gangnam = df[df['CGG_NM'] == '강남구']
#print(df_gangnam.head(1))


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [21]:
import requests

url = 'http://openapi.seoul.go.kr:8088/5345624a6b617764353756767a474c/json/tbLnOpendataRtmsV/1/5/20250301'

response = requests.get(url)
print(response.content)

b'{"RESULT":{"CODE":"INFO-200","MESSAGE":"\xed\x95\xb4\xeb\x8b\xb9\xed\x95\x98\xeb\x8a\x94 \xeb\x8d\xb0\xec\x9d\xb4\xed\x84\xb0\xea\xb0\x80 \xec\x97\x86\xec\x8a\xb5\xeb\x8b\x88\xeb\x8b\xa4."}}'


In [5]:
import pandas as pd
import requests
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# 설정
SERVICE_KEY = '5345624a6b617764353756767a474c'
BASE_URL = f'http://openapi.seoul.go.kr:8088/{SERVICE_KEY}/json/tbLnOpendataRtmsV'
NUM_PAGES = 11  # 1000건 * 11 = 최대 11,000건

def fetch_page(j):
    start_idx = 1 + (j - 1) * 1000
    end_idx = j * 1000
    url = f'{BASE_URL}/{start_idx}/{end_idx}'
    try:
        response = requests.get(url)
        content = response.json()
        rows = content['tbLnOpendataRtmsV'].get('row', [])
        return pd.DataFrame(rows)
    except Exception as e:
        print(f"[에러] 페이지 {j} 요청 실패: {e}")
        return pd.DataFrame()

results = []
with ThreadPoolExecutor(max_workers=6) as executor:
    for df in tqdm(executor.map(fetch_page, range(1, NUM_PAGES + 1)), total=NUM_PAGES):
        results.append(df)

data = pd.concat(results, ignore_index=True)

print(data.head())

100%|█████████████████████████████████████████████| 11/11 [00:31<00:00,  2.88s/it]


  RCPT_YR CGG_CD CGG_NM STDG_CD STDG_NM LOTNO_SE LOTNO_SE_NM   MNO   SNO  \
0    2025  11530    구로구   10200     구로동        1          대지  1130  0023   
1    2025  11290    성북구   13800     장위동        1          대지  0219  0314   
2    2025  11620    관악구   10100     봉천동        1          대지  1717  0000   
3    2025  11290    성북구   13800     장위동        1          대지  0230  0105   
4    2025  11305    강북구   10100     미아동        1          대지  0129  0008   

       BLDG_NM  ... THING_AMT ARCH_AREA  LAND_AREA   FLR  RGHT_SE RTRCN_DAY  \
0  골드마인(Ⅲ)오피스텔  ...      6000     14.40      16.96   3.0                      
1       더커먼즈37  ...     28500     26.85      20.00   4.0                      
2       관악푸르지오  ...     76000     84.75       0.00  14.0                      
3        동방빌리지  ...     23400     40.98      18.00  -1.0                      
4       MK프라디움  ...     51500     55.00      38.00   4.0                      

  ARCH_YR BLDG_USG DCLR_SE OPBIZ_RESTAGNT_SGG_NM  
0    2002     오피스

# 공공데이터포털

In [7]:
import requests
import xmltodict
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# 
SERVICE_KEY = 'HHeKqMrerTqMhPse7LEB80QuK4KcGddt88mTHjz4qeIONsHYQgRmcCrDiehy3CEt7a9owu2TX1Brr337xz3e2Q%3D%3D'
BASE_URL = "http://apis.data.go.kr/1613000/RTMSDataSvcAptTradeDev/getRTMSDataSvcAptTradeDev"

# 지역 코드 (서울: 11110 ~ 전국 주요 코드, 일부만 예시. 전체 원할 시 파일 제공 가능)
REGION_CODES = [
    "11110", "11200", "11305",  # 서울 일부
    "26110", "26260",           # 부산 일부
    "41110", "41170",           # 경기 일부
    "42110",                   # 강원
    "43110",                   # 충북
    "44130",                   # 충남
    "45110",                   # 전북
    "46110",                   # 전남
    "47111",                   # 대구
    "48121",                   # 경남
    "50110"                    # 제주
]

# 월 리스트
months = [f"202501", "202502", "202503", "202504", "202505", "202506"]

# 수집 함수
def fetch_data(lawd_cd, deal_ymd):
    params = {
        "serviceKey": SERVICE_KEY,
        "LAWD_CD": lawd_cd,
        "DEAL_YMD": deal_ymd,
        "numOfRows": "1000",
        "pageNo": "1"
    }
    try:
        response = requests.get(BASE_URL, params=params, timeout=10)
        data = xmltodict.parse(response.text)
        items = data['response']['body']['items']
        if not items:
            return pd.DataFrame()
        item = items['item']
        if isinstance(item, list):
            return pd.DataFrame(item)
        else:
            return pd.DataFrame([item])
    except Exception as e:
        print(f"[오류] {lawd_cd}-{deal_ymd}: {e}")
        return pd.DataFrame()

# 병렬 처리 수집
results = []
with ThreadPoolExecutor(max_workers=12) as executor:
    tasks = []
    for ym in months:
        for code in REGION_CODES:
            tasks.append((code, ym))
    for df in tqdm(executor.map(lambda x: fetch_data(*x), tasks), total=len(tasks)):
        if not df.empty:
            results.append(df)

# 데이터 병합 및 정제
final_df = pd.concat(results, ignore_index=True)
print(final_df.head())


 43%|███████████████████                         | 39/90 [00:00<00:00, 187.83it/s]

[오류] 45110-202501: 'response'
[오류] 43110-202501: 'response'
[오류] 11110-202501: 'response'
[오류] 41110-202501: 'response'
[오류] 44130-202501: 'response'
[오류] 11200-202501: 'response'
[오류] 46110-202501: 'response'
[오류] 26260-202501: 'response'
[오류] 42110-202501: 'response'
[오류] 41170-202501: 'response'
[오류] 26110-202501: 'response'
[오류] 11305-202501: 'response'
[오류] 48121-202501: 'response'
[오류] 50110-202501: 'response'
[오류] 47111-202501: 'response'
[오류] 11200-202502: 'response'
[오류] 11110-202502: 'response'
[오류] 26260-202502: 'response'
[오류] 11305-202502: 'response'
[오류] 41110-202502: 'response'
[오류] 43110-202502: 'response'
[오류] 26110-202502: 'response'
[오류] 42110-202502: 'response'
[오류] 41170-202502: 'response'
[오류] 45110-202502: 'response'
[오류] 46110-202502: 'response'
[오류] 47111-202502: 'response'
[오류] 11110-202503: 'response'
[오류] 26260-202503: 'response'
[오류] 50110-202502: 'response'
[오류] 11200-202503: 'response'
[오류] 48121-202502: 'response'
[오류] 44130-202502: 'response'
[오류] 11305

 91%|████████████████████████████████████████    | 82/90 [00:00<00:00, 202.36it/s]

[오류] 26260-202504: 'response'
[오류] 50110-202503: 'response'
[오류] 46110-202504: 'response'
[오류] 41170-202504: 'response'
[오류] 41110-202504: 'response'
[오류] 42110-202504: 'response'
[오류] 44130-202504: 'response'
[오류] 43110-202504: 'response'
[오류] 47111-202504: 'response'
[오류] 48121-202504: 'response'
[오류] 45110-202504: 'response'
[오류] 50110-202504: 'response'
[오류] 11200-202505: 'response'
[오류] 11305-202505: 'response'
[오류] 11110-202505: 'response'
[오류] 41170-202505: 'response'
[오류] 43110-202505: 'response'
[오류] 26110-202505: 'response'
[오류] 41110-202505: 'response'
[오류] 42110-202505: 'response'
[오류] 26260-202505: 'response'
[오류] 45110-202505: 'response'
[오류] 44130-202505: 'response'
[오류] 11110-202506: 'response'
[오류] 48121-202505: 'response'
[오류] 50110-202505: 'response'
[오류] 47111-202505: 'response'
[오류] 46110-202505: 'response'
[오류] 11200-202506: 'response'
[오류] 26110-202506: 'response'
[오류] 26260-202506: 'response'
[오류] 43110-202506: 'response'
[오류] 11305-202506: 'response'
[오류] 41110

100%|████████████████████████████████████████████| 90/90 [00:00<00:00, 203.13it/s]


ValueError: No objects to concatenate